# J2P Power Train

Defining some constants for the J2P Bot used in the following calculations

In [52]:
import math
import numpy as np

# J2P Constants
TOTAL_MASS     = 10    # [kg]
TOTAL_LENGTH   = 0.700 # [m]
TOTAL_WIDTH    = 0.540 # [m]
TOTAL_HEIGHT   = 0.120 # [m]

WHEEL_DIAMETER = 0.12  # [m]
WHEEL_RADIUS   = WHEEL_DIAMETER / 2.0 # [m]

# General constants
GRAVITY        = 9.810 # m/s^2

# Friction
#STATIC_FR_WHEEL_STREET  = 1.05 # Schlupf 20% Quelle: Diplomarbeit Daniel Wallner TU Graz
STATIC_FR_WHEEL_STREET  = 0.80 # Source: https://www.schweizer-fn.de/stoff/reibwerte/reibwerte.php
STATIC_FR_WHEEL_GRAVEL  = 0.45 # Schlupt 20% Quelle: Diplomarbeit Daniel Wallner TU Graz

SLIDING_FR_WHEEL_STREET = 0.50 # Sliding friction on dry concrete Source: https://www.schweizer-fn.de/stoff/reibwerte/reibwerte.php

ROLLING_FR_WHEEL_STREET = 0.015 # Rolling resistance of car tire on asphalt. Source: https://de.wikipedia.org/wiki/Rollwiderstand
ROLLING_f_WHEEL_STREET  = 0.0008 # Hebelarm der Rollreibung Source: Tabellenbuch Metall mit Formelsammlung - Digitales Buch, Europa-Nr. 10609V, 48. Auflage

# Environment
DEFAULT_SLOPE = 0 # Zero degree -> no slope
ROBOT_LOAD    = 0 # Load to the robot (e.g. electronics, battery, camera etc.)

# Global Scenarios
DEFAULT_EPSILON  = 0.1 # 10% addon because of bearing and supports
DEFAULT_VELOCITY = 2.5 # m/s
DEFAULT_ACCEL    = 0.5 # m/s^2

## Weight

Caclulating the weight, the normal weight force and also the downhill slope force

In [53]:
Total_FG = (TOTAL_MASS + ROBOT_LOAD) * GRAVITY
Total_FGN = math.cos(math.radians(DEFAULT_SLOPE)) * Total_FG
Total_FGH = math.sin(math.radians(DEFAULT_SLOPE)) * Total_FG

print("FG : {:3.4} N".format(Total_FG))
print("FGN: {:3.4} N".format(Total_FGN))
print("FGH: {:3.4} N".format(Total_FGH))

FG : 98.1 N
FGN: 98.1 N
FGH: 0.0 N


Calculating the weight and normal force for a single wheel

In [54]:
Wheel_FGN = (math.cos(math.radians(DEFAULT_SLOPE)) * Total_FG) / 6
Wheel_FGH = (math.sin(math.radians(DEFAULT_SLOPE)) * Total_FG) / 6

print("FGN: {:3.4} N".format(Wheel_FGN))
print("FGH: {:3.4} N".format(Wheel_FGH))

FGN: 16.35 N
FGH: 0.0 N


## Friction

### Static Friction

The first calculation covers the static friction. For this we use only friction factors for car tires on asphalt

In [55]:
# Defining the MU_H values
MU_H = STATIC_FR_WHEEL_STREET # Friction factor of car tire on dry asphalt
#MU_H = 0.5 # Friction factor of car tire on wet asphalt
#MU_H = 0.1 # Friction factor of car tire on ice

Wheel_FR = MU_H * Wheel_FGN

print("Wheel_FR: {:3.4} N".format(Wheel_FR))

Wheel_FR: 13.08 N


### Sliding Friction

The calculation of the sliding friction is nearly the same as for the static friction. Only the friction factor changes

In [56]:
# Defining the MU_G values
MU_G = SLIDING_FR_WHEEL_STREET # Sliding Friction factor of car tire on dry asphalt

Wheel_FGR = MU_G * Wheel_FGN

print("Wheel_FGR: {:3.4} N".format(Wheel_FGR))

Wheel_FGR: 8.175 N


### Rolling Friction
The roling friction calculation is from the mathematical point of view very similar to the other friction types. But the friction factor is completely different and is based on completely different physical concepts

In [57]:
# Defining the cR value
CR = ROLLING_f_WHEEL_STREET / WHEEL_RADIUS
#CR = ROLLING_FR_WHEEL_STREET # rolling friction factor of car tire on dry asphalt

Wheel_FRR = CR * Wheel_FGN

print("Wheel_FRR: {:3.4} N".format(Wheel_FRR))

Wheel_FRR: 0.218 N


## Movement Scenarios

### Scenario A: Movement on flat, horizontal surface

In [73]:
# The pure force for the theoretical movement without resistances
Wheel_FA = (Wheel_FGN / GRAVITY) * DEFAULT_ACCEL
# The force including the resistances (rolling resistance and downhill slope force)
Wheel_FA_Real = Wheel_FA + Wheel_FRR + Wheel_FGH
# Add the bearing and support losses
Wheel_FA_Real = Wheel_FA_Real + (Wheel_FA_Real * DEFAULT_EPSILON)

# Check whether the force is smaller or equal to FR
NotSlipping = Wheel_FA <= Wheel_FR

# Calculate the needed torque
Wheel_MA = Wheel_FA_Real * WHEEL_RADIUS

print("Wheel_FA     : {:3.4} N".format(Wheel_FA))
print("Wheel_FA_Real: {:3.4} N".format(Wheel_FA_Real))
print("Not Slipping : {}".format(NotSlipping))

print("Wheel_MA     : {:3.4} Nm".format(Wheel_MA))

Wheel_FA     : 0.8333 N
Wheel_FA_Real: 10.15 N
Not Slipping : True
Wheel_MA     : 0.6089 Nm


### Scenario B: Movement across a small obstacle

In [75]:
# First, defining the obstacle
Obstacle_Height = 0.03 # 3cm height

# Now calculating the angle phi in the wheel circle
cosPhi = (WHEEL_RADIUS - Obstacle_Height) / WHEEL_RADIUS # (r - h) / r
phi = math.degrees(math.acos((cosPhi)))
# Calculating the distance y
y = math.sin(math.radians(phi)) * WHEEL_RADIUS
# Calculating the angle alpha for the slope
alpha = math.degrees(math.atan(Obstacle_Height / y))

print("Angle Phi         : {:3.4}°".format(phi))
print("Distance y        : {:3.4}m ({:3.4}cm)".format(y, y*100))
print("Angle Alpha       : {:3.4}°".format(alpha))

# Calculate the downhill slope force for the obstacle (based on alpha)
Wheel_FGH_Obstacle = (math.sin(math.radians(alpha)) * Total_FG) / 6
print("Wheel_FGH_Obstacle: {:3.4}N\n".format(Wheel_FGH_Obstacle))

# The pure force for the theoretical movement without resistances
Wheel_FA = (Wheel_FGN / GRAVITY) * (DEFAULT_ACCEL / 4.0) # Use only a 4th of the acceleration
# The force including the resistances (rolling resistance and downhill slope force + downhill clopse force obstacle)
Wheel_FA_Real = Wheel_FA + Wheel_FRR + Wheel_FGH + Wheel_FGH_Obstacle
# Add the bearing and support losses
Wheel_FA_Real = Wheel_FA_Real + (Wheel_FA_Real * DEFAULT_EPSILON)

# Check whether the force is smaller or equal to FR
NotSlipping = Wheel_FA <= Wheel_FR

# Calculate the needed torque
Wheel_MA = Wheel_FA_Real * WHEEL_RADIUS

print("Wheel_FA     : {:3.4} N".format(Wheel_FA))
print("Wheel_FA_Real: {:3.4} N".format(Wheel_FA_Real))
print("Not Slipping : {}".format(NotSlipping))

print("Wheel_MA     : {:3.4} Nm".format(Wheel_MA))

Angle Phi         : 60.0°
Distance y        : 0.05196m (5.196cm)
Angle Alpha       : 30.0°
Wheel_FGH_Obstacle: 8.175N

Wheel_FA     : 0.2083 N
Wheel_FA_Real: 18.45 N
Not Slipping : True
Wheel_MA     : 1.107 Nm
